In [1]:
%matplotlib notebook
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt

In [2]:
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exp_date = '2017_11_26'
exp_name = '2017_11_26_16_04_49_readout-timing-scan'
experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)

name 'fort_exp' is not defined


In [3]:
q.experiments[0]['variable_list']

[u'mot_timing_offset']

In [12]:
sig = []
for i in q.experiments[0]['iterations']:
    results = q.generate_thresholds(itr=i)[0]
    sig.append(results['fit_params'][3]-results['fit_params'][2])

In [14]:
sig[4]

30.757324792530476